In [36]:
import multiprocessing
import os
from pathlib import Path

import matplotlib.pyplot as plt
import nltk
import pandas as pd
import seaborn as sns
from gensim.models import Word2Vec

In [28]:
pd.set_option('display.max_colwidth', None)

main_path = Path.cwd() / ".."
songs_list = os.listdir(main_path / "data/")

Vamos a utilizar como corpus la letra de las cacniones de Atahualpa Yupanqui, descargadas del sitio web [letras.com](https://www.letras.com/atahualpa-yupanqui/), utilizando la tecnica de web scraping.

Como documentos seleccionamos cada uno de los parrafos dentro de las canciones, esto es, estrofas y estribillos. Esto debido a que, por lo general, en las canciones de folclore cada parrafo se encuentra contextualizado en si mismo. Ademas, si tomaramos cada verso como corpus encontrariamos que algunas canciones poseen versos de tan solo algunas palabras.

In [ ]:
# load the documents.
list_dfs = []
for song in songs_list:
    try:
        list_dfs.append(pd.read_csv(main_path / f"data/{song}", sep='/n', header=None, engine='python'))
    except Exception as e:
        print(f"Error: couldn't load {song}")

df = pd.concat(list_dfs)

Error: couldn't load Cancion Del Abuelo N 2.txt
Error: couldn't load Danza de La Luna.txt
Error: couldn't load Danza Santiaguena.txt
Error: couldn't load Don Fermin.txt
Error: couldn't load El Mal Dormido.txt
Error: couldn't load El Rescoldeao.txt
Error: couldn't load Estilo Serrano.txt
Error: couldn't load Gato Santiagueno.txt
Error: couldn't load La Estancia Vieja.txt
Error: couldn't load La Humilde.txt
Error: couldn't load La Nadita.txt
Error: couldn't load Malquistao o Vidala Dolorosa.txt
Error: couldn't load Melodia Del Adios y Danza Rustica.txt
Error: couldn't load Milonga triste.txt
Error: couldn't load Paso de Los Andes.txt
Error: couldn't load Vidala.txt
Error: couldn't load Zamba Tucumana.txt


In [21]:
print(f"Cantidad de canciones en el corpus: {len(list_dfs)}")

Cantidad de canciones en el corpus: 283


In [30]:
# Visualize the first song
list_dfs[0].head(5)

,0
0,Estaba el Cerro tranquilo Cada cual en su trabajo Cuando llegaron de abajo Tres democratas eternos Emisarios del Gobierno La lengua como badajo
1,Reunir a la paisanada Traian como mision Para hacer una funcion De asados y de empanadas Y asi tomar posesion De toda gruta pintada
2,"Se busco primeramente La sombra de Arganaras Pero dada la humedad Y del mucho genterio En los talas, junto al rio Se vido mas ampliedad"
3,"Para sacar el yuyal Con hachas, palas y picos Trajeron muchos melicos De los tres Departamentos Y a los pobres, al momento Les humeaban los hocicos"
4,Lo que antes fue matorral Quedo mesmo que un salon Regadito y parejon Y medio atras del ramaje Un bano pa'l mujeraje Con forma de corazon


In [ ]:
print(f"Cantidad de documentos en el corpus: {df.shape[0]}")

Cantidad de documentos en el corpus: 2187


#### 1. Preprocesamiento

In [32]:
from tensorflow.keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

In [35]:
# Demos un vistazo
sentence_tokens[0]

['estaba',
 'el',
 'cerro',
 'tranquilo',
 'cada',
 'cual',
 'en',
 'su',
 'trabajo',
 'cuando',
 'llegaron',
 'de',
 'abajo',
 'tres',
 'democratas',
 'eternos',
 'emisarios',
 'del',
 'gobierno',
 'la',
 'lengua',
 'como',
 'badajo']

In [128]:
import re

# Remove stopwords
stopwords_es = nltk.corpus.stopwords.words("spanish")
sentence_tokens_nostop = [
    [
        re.sub(r"[^a-zA-Z0-9 ]", "", word)
        for word in document
        if word not in stopwords_es
    ]
    for document in sentence_tokens
]

# Remove words with less than 3 letters
sentence_tokens_nostop = [
    [
        word
        for word in document
        if len(word) >= 3
    ]
    for document in sentence_tokens_nostop
]

#### 2. Crear los vectores Word2Vec

In [129]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [218]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=7,       # cant de palabras antes y desp de la predicha
                     vector_size=32,       # dimensionalidad de los vectores 
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [219]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens_nostop)

In [220]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 2187


In [221]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de words distintas en el corpus: 1024


### 3. Entrenar embeddings

In [222]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=100,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 399911.15625
Loss after epoch 1: 287933.03125
Loss after epoch 2: 286025.9375
Loss after epoch 3: 257787.75
Loss after epoch 4: 246558.75
Loss after epoch 5: 242733.5
Loss after epoch 6: 237808.875
Loss after epoch 7: 223285.0
Loss after epoch 8: 205693.25
Loss after epoch 9: 200170.0
Loss after epoch 10: 198766.25
Loss after epoch 11: 196974.75
Loss after epoch 12: 195877.75
Loss after epoch 13: 192107.25
Loss after epoch 14: 190335.75
Loss after epoch 15: 190683.5
Loss after epoch 16: 188723.25
Loss after epoch 17: 188047.25
Loss after epoch 18: 179028.5
Loss after epoch 19: 174673.5
Loss after epoch 20: 173017.0
Loss after epoch 21: 175027.5
Loss after epoch 22: 173321.5
Loss after epoch 23: 173010.0
Loss after epoch 24: 171464.5
Loss after epoch 25: 172679.0
Loss after epoch 26: 173595.5
Loss after epoch 27: 171144.5
Loss after epoch 28: 171295.5
Loss after epoch 29: 171400.0
Loss after epoch 30: 169910.5
Loss after epoch 31: 170882.0
Loss after epoch 32: 171257

(1606650, 4642400)

### 4. Visualizar similaridades

In [223]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["bombo"], topn=10)

[('hondura', 0.7071337699890137),
 ('musico', 0.6913033127784729),
 ('llamando', 0.6702632904052734),
 ('violin', 0.6505707502365112),
 ('campo', 0.6426671743392944),
 ('tum', 0.6257089376449585),
 ('cuerdas', 0.624566912651062),
 ('caja', 0.6136906743049622),
 ('marca', 0.6087656021118164),
 ('mama', 0.6067737340927124)]

Observamos que entre las palabras con mayor similaridad hay algunas relacionadas con la musica: "musico", "violin", "cuerdas", "caja", y algunas tambien relacionados con acciones a las que se asocia el bombo leguero en las canciones folcloricas: "llamando", "marca".

In [224]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["caballo"], topn=10)

[('solo', 0.7228654623031616),
 ('tira', 0.7127429246902466),
 ('montiel', 0.7030181288719177),
 ('adios', 0.6221621632575989),
 ('flete', 0.6213982701301575),
 ('adelante', 0.6156866550445557),
 ('empieza', 0.6148142218589783),
 ('paisanos', 0.6043440699577332),
 ('corral', 0.5963303446769714),
 ('vale', 0.5917187333106995)]

Observamos que entre las palabras con mayor similaridad se encuentran terminos que contextualizan a la vision del caballo en las canciones de Yupanqui: "tira el caballo adelante..." en la cancion "La Aniera" es un ejemplo. Observamos que aparece la palabra "corral", logicamente relacionada con "caballo".

In [225]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["zamba"], topn=10)

[('changos', 0.6827086806297302),
 ('bailando', 0.6682401299476624),
 ('panuelo', 0.6595569252967834),
 ('baila', 0.6516932249069214),
 ('palomitay', 0.6391434669494629),
 ('perdida', 0.6320529580116272),
 ('zaratena', 0.6272075772285461),
 ('pagos', 0.626753568649292),
 ('travieso', 0.6256049275398254),
 ('lujan', 0.6255539655685425)]

Observamos que la palabra "zamba" se encuentra muy referenciada en cuanto al baile en las canciones de Yupanqui: "bailando", "panuelo", "baila" aparecen entre las 4 mas similares.

In [226]:
w2v_model.wv.most_similar(positive=["noche"], topn=5)

[('adentro', 0.7101600766181946),
 ('pena', 0.6990299224853516),
 ('canto', 0.6923782229423523),
 ('luna', 0.6917726993560791),
 ('guitarra', 0.6693127751350403)]

In [227]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(negative=["noche"], topn=5)

[('lalma', 0.11442771553993225),
 ('zaino', 0.001967464806511998),
 ('india', -0.028786981478333473),
 ('padir', -0.048010725528001785),
 ('saco', -0.05696024000644684)]

Observamos la diferencia que hay entre las palabras mas cercanas a noche y las mas lejanas utilizando la similaridad coseno. Entre las mas cecanas encontramos a "luna", "canto", "pena" y "guitarra", ademas encontramos "adentro", palabra que suele ir junto a "noche" en las canciones de Yupanqui. Por otro lado, "lalma" (el alma), "zaino", "saco" se encuentran entre las mas disimiles.

Es importante considerar que, debido a la eleccion de una ventana de 7 palabras, el algoritmo va a tender a conectar palabras que suelen aparecer juntas en las canciones. Esto debido a que al aparecer juntas comparten vecinos.

#### 5. Buscar asociaciones

In [270]:
w2v_model.wv.most_similar(positive=["lejos"], topn=5)

[('alto', 0.7003306746482849),
 ('viento', 0.6940330266952515),
 ('pena', 0.6855090856552124),
 ('preciso', 0.6759711503982544),
 ('aquel', 0.6291369795799255)]

In [268]:
words = ["rancho", "cerca", "lejos"]

vector_1 = w2v_model.wv.get_vector(words[0])
vector_2 = w2v_model.wv.get_vector(words[1])
vector_3 = w2v_model.wv.get_vector(words[2])

new_word = vector_1 - vector_2 + vector_3

In [269]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Extract all word embeddings and corresponding words in the model
words_w2v = list(w2v_model.wv.index_to_key)  # List of words in the Word2Vec vocabulary
embeddings_w2v = np.array([w2v_model.wv[word] for word in words_w2v])  # Convert embeddings to numpy array

# Calculate cosine similarity between new_word_vector and all other embeddings
similarities = np.squeeze(cosine_similarity(embeddings_w2v, [new_word]))

# Sort and get the top 10 most similar words
args_sorted = np.argsort(similarities)[::-1][:10]
for arg in args_sorted:
    print(f'{words_w2v[arg]}: {similarities[arg]}')

lejos: 0.6415697336196899
espina: 0.6338721513748169
adios: 0.6307780742645264
feliz: 0.624732494354248
algun: 0.6065328121185303
tucumana: 0.592726469039917
sendas: 0.5728856325149536
ninguno: 0.5714237689971924
pago: 0.5707380771636963
noche: 0.5678344964981079


La busqueda de asociaciones no permite demasiada flexibilidad, pero eso se debe probablemente al reducido tamanio de nuestro corpus.

Podemos observar que al restar el vector correspondiente a "cerca" de la palabra "rancho", y sumar la palabra "lejos" al resultado, obtenemos palabras como "espina", "adios", "sendas" a las que podemos llegar a asociar al sentimiento de estar lejos de casa. Sin embargo no podemos estar seguros si esa asociacion es causal o no.

#### Visualizar vectores

In [278]:
from sklearn.decomposition import IncrementalPCA    
from sklearn.manifold import TSNE                   
import numpy as np                                  

def reduce_dimensions(model, num_dimensions = 2 ):
     
    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)  

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [279]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

vecs, labels = reduce_dimensions(w2v_model)

MAX_WORDS = 200
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show() # renderer="colab" esto para plotly en colab

In [281]:
# Graficar los embedddings en 3D

vecs, labels = reduce_dimensions(w2v_model,3)

fig = px.scatter_3d(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], z=vecs[:MAX_WORDS,2],text=labels[:MAX_WORDS])
fig.update_traces(marker_size = 2)
fig.show() # renderer="colab" esto para plotly en colab

Podemos observar ka cercania entre algunos de los terminos como ("hoy", "maniana"), ("cantor", "cancion"), y ("vengo", "vuelve") por ejemplo. Resultaria interesante realizar el mismo procedmiento pero con un corpus mayor, quizas incluyendo las canciones de multiples artistas y no solo las de Atahualpa Yupanqui. Eso podria contribuir a la variacion en el contexto de utilizacion de los terminos permitiendo contextualizar mejor cada una de las palabras.